## Dependencies

In [13]:
import os.path as osp
import glob
import cv2
import numpy as np
import torch
import ESRGAN.RRDBNet_arch as arch # Make sure you clone the ESRGAN repository
import os
from tkinter import Tk, filedialog
import matplotlib.pyplot as plt


model_path = os.path.join(os.getcwd(),'ESRGAN','models','RRDB_ESRGAN_x4.pth') # change this if incorrect
print(model_path)


c:\Projects\python\image-video-super-resolution\ESRGAN\models\RRDB_ESRGAN_x4.pth


## Device setup

In [14]:
if torch.cuda.is_available():    
    device = torch.device("cuda")
    current_device= torch.cuda.current_device()
    # Limit memory usage to 80%
    torch.cuda.set_per_process_memory_fraction(0.8,device=current_device)
    # For optimized memory utilization during tensor operations
    torch.backends.cudnn.benchmark = True
    print(f"Using Device: {device}",
          f"\nCurrent GPU: {torch.cuda.get_device_name(current_device)}",
          f"\nCuda version: {torch.version.cuda}",
          f"\ncuDNN available: {torch.backends.cudnn.is_available()}",
          f"\ncuDNN version: {torch.backends.cudnn.version()}",
          f"\nAllocated memory: {torch.cuda.memory_allocated()} bytes",
          f"\nCached memory: {torch.cuda.memory_reserved()} bytes")
else:
    device = torch.device("cpu")
    print(f"Using Device: {device}")

Using Device: cuda 
Current GPU: NVIDIA GeForce RTX 4060 Laptop GPU 
Cuda version: 11.8 
cuDNN available: True 
cuDNN version: 90100 
Allocated memory: 0 bytes 
Cached memory: 0 bytes


In [15]:
# load the model
model = arch.RRDBNet(in_nc=3,out_nc=3, nf=64,nb=23,gc=32)
model.load_state_dict(torch.load(model_path),strict=True)
model.eval()
model = model.to(device=device)

C:\Users\harsh\AppData\Local\Temp\ipykernel_9116\3631384658.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path),strict=True)


## Main

In [ ]:
test_img_folder = 'LR/*'

model = arch.RRDBNet(3, 3, 64, 23, gc=32)
model.load_state_dict(torch.load(model_path), strict=True)
model.eval()
model = model.to(device)

print('Model path {:s}. \nTesting...'.format(model_path))

idx = 0
for path in glob.glob(test_img_folder):
    idx += 1
    base = osp.splitext(osp.basename(path))[0]
    print(idx, base)
    # read images
    img = cv2.imread(path, cv2.IMREAD_COLOR)
    img = img * 1.0 / 255
    img = torch.from_numpy(np.transpose(img[:, :, [2, 1, 0]], (2, 0, 1))).float()
    img_LR = img.unsqueeze(0)
    img_LR = img_LR.to(device)

    with torch.no_grad():
        output = model(img_LR).data.squeeze().float().cpu().clamp_(0, 1).numpy()
    output = np.transpose(output[[2, 1, 0], :, :], (1, 2, 0))
    output = (output * 255.0).round()
    cv2.imwrite('results/{:s}_rlt.png'.format(base), output)
